### Importing Libraries

In [1]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential

### Workspace Configuration

In [2]:
subscription_id = "0ee863a6-4590-48f5-8af7-559cc741a2ae"
resource_group = "product_detection"
workspace = "az_deployment"

In [3]:
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id = subscription_id,
    resource_group = resource_group,
    workspace = workspace
)

### Endpoint Configuration

In [4]:
import datetime
endpoint_name = "endpt-mdl2"+ datetime.datetime.now().strftime("%m%d%H%M%f")
endpoint = ManagedOnlineEndpoint(
    name = endpoint_name,
    description = "Test Endpoint",
    auth_mode = "key"
)

### Deployment Configuration

In [5]:
model = Model(path="best.pt")
env = Environment(
    conda_file="conda.yaml",
    image="umairsiddique3171/azlocaldeployment:lastest"
)
test_deployment = ManagedOnlineDeployment(
    name="test",
    endpoint_name=endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code=".", scoring_script="score.py"
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

### Local Deployment

In [6]:
ml_client.online_endpoints.begin_create_or_update(endpoint=endpoint,local=True)     ### local docker deployment for final testing

Creating local endpoint (endpt-mdl210031054812399) .Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'endpt-mdl210031054812399', 'description': 'Test Endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('C:/Users/US593/.azureml/inferencing/endpt-mdl210031054812399'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001F90738AF40>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [7]:
ml_client.online_deployments.begin_create_or_update(deployment=test_deployment,local=True)   # local docker deployment for final testing

Creating local deployment (endpt-mdl210031054812399 / test) .
Building Docker image from Dockerfile
Step 1/6 : FROM umairsiddique3171/azlocaldeployment:lastest
 ---> 8ebf443213a4
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Running in a59403dcf6e7
 ---> 4e78a1a9475b
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Running in a6bf0f0fca70
 ---> 664ac3d1ba0f
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> 87d939d85a70
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Running in 0621d0953e87
.Retrieving notices: ...working... done
Channels:
 - defaults
 - conda-forge
Platform: linux-64
Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
    current version: 24.7.1
    latest version: 24.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda


......
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Installing pip dependencie

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpt-mdl210031054812399', 'type': 'Managed', 'name': 'test', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('c:/Users/US593/OneDrive/Desktop/yolov5_local_deployment/src'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001F90751C700>, 'model': Model({'job_name': None, 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': '346c41277a884a99e750425df542fe3f', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('c:/Users/US593/OneDrive/Desktop/yolov5_local_deployment/src'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001F90

### Local Deployment Status

In [8]:
ml_client.online_endpoints.get(name=endpoint_name,local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Succeeded', 'scoring_uri': 'http://localhost:32770/score', 'openapi_uri': None, 'name': 'endpt-mdl210031054812399', 'description': 'Test Endpoint', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': WindowsPath('c:/Users/US593/OneDrive/Desktop/yolov5_local_deployment/src'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001F9075420A0>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

### Scoring Local Endpoint

In [9]:
import requests
import json
import base64
from PIL import Image

endpoint_url = "http://localhost:32770/score"

def load_image(image_path):
    with open(image_path, "rb") as img_file:
        img_bytes = img_file.read()
    return base64.b64encode(img_bytes).decode("utf-8")

def invoke_local_endpoint(image_path):
    base64_image = load_image(image_path)
    payload = json.dumps({
        "data": base64_image
    })
    headers = {
        'Content-Type': 'application/json'
    }

    try:
        response = requests.post(endpoint_url, headers=headers, data=payload)

        if response.status_code == 200:
            result = response.json()
            return result
        else:
            print(f"Error: {response.status_code} - {response.text}")
    except Exception as e:
        print(f"Failed to invoke endpoint: {str(e)}")

image_path = "test.jpg"
predictions = invoke_local_endpoint(image_path)
print(predictions)


{"predictions": [{"box": [1886.4769287109375, 2139.99609375, 307.462646484375, 792.9683837890625], "confidence": 0.886823832988739, "class": "object"}, {"box": [1513.69287109375, 756.2691650390625, 356.634765625, 669.17724609375], "confidence": 0.8736401200294495, "class": "object"}, {"box": [144.21156311035156, 2918.751708984375, 284.5641174316406, 210.49658203125], "confidence": 0.8730153441429138, "class": "object"}, {"box": [499.4511413574219, 2918.05908203125, 394.48651123046875, 208.565185546875], "confidence": 0.8723738193511963, "class": "object"}, {"box": [3098.718017578125, 2393.12158203125, 741.33251953125, 302.715087890625], "confidence": 0.8659147620201111, "class": "object"}, {"box": [903.991455078125, 2920.61474609375, 377.32672119140625, 204.032470703125], "confidence": 0.8602424263954163, "class": "object"}, {"box": [114.35187530517578, 2231.680908203125, 228.70375061035156, 583.1705322265625], "confidence": 0.8599406480789185, "class": "object"}, {"box": [786.33557128